In [1]:
import pandas as pd
import seaborn as sns
import tensorflow.keras
import shap
import tensorflow as tf

#let's load the diamonds dataset
df=sns.load_dataset(name='diamonds')
print(df.head())

   carat      cut color clarity  depth  table  price     x     y     z
0   0.23    Ideal     E     SI2   61.5   55.0    326  3.95  3.98  2.43
1   0.21  Premium     E     SI1   59.8   61.0    326  3.89  3.84  2.31
2   0.23     Good     E     VS1   56.9   65.0    327  4.05  4.07  2.31
3   0.29  Premium     I     VS2   62.4   58.0    334  4.20  4.23  2.63
4   0.31     Good     J     SI2   63.3   58.0    335  4.34  4.35  2.75


In [2]:
import numpy as np

In [3]:
from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [6]:
X = df

In [7]:
# get target and feature variables

#set them as categorical
X[['cut','color','clarity']] = X[['cut','color','clarity']].astype('category')

# get the maps to the encoding
cut_cat_map = [x for x in zip(X.cut.cat.categories,X.cut.cat.codes.unique() )]
color_cat_map = [x for x in zip(X.color.cat.categories,X.cut.cat.codes.unique() )]
clarity_cat_map = [x for x in zip(X.clarity.cat.categories,X.cut.cat.codes.unique() )]

# replace categorical classes to their encoding
X[['cut','color','clarity']] = X[['cut','color','clarity']].apply(lambda c : c.cat.codes)

print('cut_cat_map:' , cut_cat_map)
print('color_cat_map: ',color_cat_map)
print('clarity_cat_map: ', clarity_cat_map)

cut_cat_map: [('Ideal', 0), ('Premium', 1), ('Very Good', 3), ('Good', 2), ('Fair', 4)]
color_cat_map:  [('D', 0), ('E', 1), ('F', 3), ('G', 2), ('H', 4)]
clarity_cat_map:  [('IF', 0), ('VVS1', 1), ('VVS2', 3), ('VS1', 2), ('VS2', 4)]


In [8]:
X

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,0,1,6,61.5,55.0,326,3.95,3.98,2.43
1,0.21,1,1,5,59.8,61.0,326,3.89,3.84,2.31
2,0.23,3,1,3,56.9,65.0,327,4.05,4.07,2.31
3,0.29,1,5,4,62.4,58.0,334,4.20,4.23,2.63
4,0.31,3,6,6,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,0,0,5,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,3,0,5,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,2,0,5,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,1,4,6,61.0,58.0,2757,6.15,6.12,3.74


In [9]:
X.to_csv("data/diamonds_num.csv", sep=',', encoding='utf-8', index=False)

In [ ]:
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

# set input layer
inputs = Input(shape=(X_train.shape[1],), name='input')

# normalized the batches
x = BatchNormalization(name='input_bn')(inputs)

# add the fully connected layers
x = Dense(X_train.shape[1], activation='relu', name='first')(x)
x = Dense(64, activation='relu',name='second')(x)
x = Dense(X_train.shape[1], activation='elu',name='last')(x)

# get the final result
predictions = Dense(1, activation='relu', name='ouput')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam',
              loss='mape')
model.fit(X_train,
          Y_train,
          epochs = 100,
          batch_size = 1000,
          validation_data = (X_test,Y_test),
          shuffle=True,
          callbacks=[EarlyStopping(monitor='val_loss', patience=1,)])


#let's get the training and validation histories for plotting
val_loss = model.history.history['val_loss']
loss = model.history.history['loss']

print(model.summary())
# let's plot the performance curve
import matplotlib.pyplot as plt
plt.figure()
plt.plot(val_loss, label='val_loss')
plt.plot(loss, label = 'loss')
plt.legend()
plt.show()

In [ ]:

# separate train/test set
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 5)
print('Y_train: ', Y_train.shape)
print('Y_test: ', Y_test.shape)

In [ ]:
import shap

#initialize js methods for visualization
shap.initjs()

# create an instance of the DeepSHAP which is called DeepExplainer
explainer_shap = shap.DeepExplainer(model=model, data=X_train.values)

# Fit the explainer on a subset of the data (you can try all but then gets slower)
shap_values = explainer_shap.shap_values(X=X_train.values[:500], ranked_outputs=True)

In [ ]:
# now let's inspect some individual explanations inferred by DeepSHAP
shap.force_plot(explainer_shap.expected_value,
                shap_values[0][0],
                feature_names=X_train.columns)

shap.force_plot(explainer_shap.expected_value,
                shap_values[0][0][1],
                X_train.values[:500][0],
                feature_names=X_train.columns,)

shap.force_plot(explainer_shap.expected_value,
                shap_values[0][0][1],
                X_train.values[:500][0],
                feature_names=X_train.columns,)

In [ ]:

# to get the output value and base value
record = 1 # this is just to pick one record in the dataset 
base_value = explainer_shap.expected_value
output= base_value + np.sum(shap_values[0][0][record])
print('base value: ',base_value)
print('output value: ',output)

#sanity check that the ouput value is equal to the actual prediction
print(np.round(output,decimals=1) == np.round(model.predict(X_train.values)[record],decimals=1))


# to get the shape values or each feature
shap_df = pd.DataFrame(list(dict(zip(X_train.columns.values,base_value)).items()),
             columns=['features','shapvals']).sort_values(by='shapvals', ascending=True)
print(shap_df)

In [ ]:

# get the ovearall mean contribution of each feature variable
shap.summary_plot(shap_values[0], X_train.values, feature_names=X_train.columns)